In [ ]:
import os

import numpy as np

import season1.statistician as stat
import season1.utility as util
from season1.validater import get_label

def transform_data_block_to_compared(data_block):
    return {
        "map": data_block["map"],
        "example_maps": data_block["examples"],
        "before_params": data_block["target_params"],
        "after_params": get_label(data_block["map"])
    }

def generate_filename_from_used_paramname(used_paramname, recursion_count):
    filename = ""
    for param in used_paramname:
        filename += param
        filename += "_"
    filename += str(recursion_count)
    return filename

def calc_controllability(compared_list, used_paramname):
    controllability = {}
    
    for paramname in used_paramname:
        values = []
        
        for compared in compared_list:
            before = compared["before_params"][paramname]
            after = compared["after_params"][paramname]
            values.append(abs(before - after) / max(before, after))
        
        controllability[paramname] = np.average(values)
    
    return controllability
            

In [ ]:
result_dir = os.path.join("..", "data", "result")
paramname_list = ["map_size", "room_count", "enemy_count", "treasure_count", "exploration", "winding_path"]

In [ ]:
controllability_data = [{}] * 6

for i in range(6):
    for paramname in paramname_list:
        used_paramname = [paramname]
        filename = generate_filename_from_used_paramname(used_paramname, i)
        
        data_blocks = util.read_json_file(os.path.join(result_dir, filename))
        compared_list = map(transform_data_block_to_compared, data_blocks)
        controllability_data[i][filename] = calc_controllability(compared_list, used_paramname)
        